
# content
1. connecting to POSTGRES
2. creating the db schema
3. testing bitemporal crud


## 1. connecting to POSTGRES

In [ ]:
using Pkg
Pkg.add(url="https://github.com/Kaeptenblaubaer/SearchLightPostgreSQL.jl")
ENV["GENIE_ENV"] = "dev"
using SearchLight
using SearchLightPostgreSQL
SearchLight.Configuration.load() |> SearchLight.connect


## creating the migration table

In [ ]:
SearchLight.Migrations.create_migrations_table()

In [ ]:
Pkg.add("TimeZones")
using TimeZones
SearchLight.Migrations.up()

## 4. Testing

In [28]:
using Test
using BitemporalPostgres, SearchLight

  t=TestDummyComponent()
  tr=TestDummyComponentRevision(description="blue")
  SearchLight.Configuration.load() |> SearchLight.connect
  
  @test BitemporalPostgres.createEntity(t,tr) != Nothing

  @test SearchLight.query("""
    SELECT i.* FROM histories h JOIN versions v ON v.ref_history = h.id
    JOIN validityIntervals i ON i.ref_version = v.id 
    JOIN testdummyComponents p ON p.ref_history = h.id
    JOIN testdummycomponentrevisions r ON r.ref_component = p.id AND r.ref_valid @> v.id 
    WHERE h.id = 1 and p.id = 1 AND  i.tsrworld @> TIMESTAMPTZ '2014-05-30 20:00:00+00' AND i.tsrdb @>  NOW() 
""") != Nothing


┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133


┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133


┌ Info: INSERT INTO validityintervals ( "ref_history", "ref_version", "tsworld_validfrom", "tsworld_invalidfrom", "tsdb_validfrom", "tsdb_invalidfrom", "is_committed" ) VALUES ( E'1', E'1', E'2014-05-30T21:00:00+01:00', E'2038-01-19T14:07:00+00:00', E'2022-01-30T18:28:24.061+01:00', E'2038-01-19T14:07:00+00:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133
NOTICE:  NEW: (1,1,1,"2014-05-30 20:00:00+00","2038-01-19 14:07:00+00","2022-01-30 17:28:24.061+00","2038-01-19 14:07:00+00",,,0)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."tsdb_validfrom" AS "validit

┌ Info: INSERT INTO testdummycomponents ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133
┌ Info: SELECT "testdummycomponents"."id" AS "testdummycomponents_id", "testdummycomponents"."ref_history" AS "testdummycomponents_ref_history" FROM "testdummycomponents" WHERE "id" = 1 ORDER BY testdummycomponents.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133


┌ Info: INSERT INTO testdummycomponentrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description" ) VALUES ( E'1', E'1', E'9223372036854775807', E'blue' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133
NOTICE:  NEW: (1,1,1,9223372036854775807,,blue)
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_invalidfrom" AS "testdummycomponentrevisions_ref_invalidfrom", "testdummycomponentrevisions"."description" AS "testdummycomponentrevisions_description" FROM "testdummycomponentrevisions" WHERE "id" = 1 ORDER BY testdummycomponentrevisions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/Search

┌ Info:     SELECT i.* FROM histories h JOIN versions v ON v.ref_history = h.id
│     JOIN validityIntervals i ON i.ref_version = v.id 
│     JOIN testdummyComponents p ON p.ref_history = h.id
│     JOIN testdummycomponentrevisions r ON r.ref_component = p.id AND r.ref_valid @> v.id 
│     WHERE h.id = 1 and p.id = 1 AND  i.tsrworld @> TIMESTAMPTZ '2014-05-30 20:00:00+00' AND i.tsrdb @>  NOW() 
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/pumJS/src/SearchLightPostgreSQL.jl:133


Test Passed